In [7]:
## Implementing the MLP character level model for words by following this paper  https://www.jmlr.org/papers/volume3/bengio03a/bengio03a.pdf
## This is the first paper introduced for language modelling with MLP. Let's implement this 

print(
"""
Roughly this paper states that finding the next character using the previous n characters. 

For Example: n=3, xs = "abc", ys = "d" 
             n=5, xs = "abcde", ys = "f" 
"""
)


Roughly this paper states that finding the next character using the previous n characters. 

For Example: n=3, xs = "abc", ys = "d" 
             n=5, xs = "abcde", ys = "f" 



In [2]:
import torch 
import torch.nn.functional as F 
import matplotlib.pyplot as plt

In [5]:
## exploring the dataset 
words = open("names.txt", "r").read().splitlines() 

print(f"Len of words: {len(words)}") 
print(f"Words: {words[:10]}")

## Unique characters
chars = sorted(list(set("".join(words)))) 
print(f"Len of unique characters: {len(chars)}") 

## building encoder, decoder like label2id and id2label 
stoi = { v:k+1 for k, v in enumerate(chars)}
stoi['.'] = 0
itos = { v:k for k, v in stoi.items()}
print(f'Itos:\n{itos}')

Len of words: 32033
Words: ['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia', 'harper', 'evelyn']
Len of unique characters: 26
Itos:
{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [112]:
# Building the dataset 
block_size = 3  # Total number of previous characters to predict the next character 

X, Y = [], []  # X is the input to the neural net, Y is the output of the corresponding X to the neural net 
for w in words[:5]: 

    context = [0] * block_size  # if n = 3 [0, 0, 0]
    print(w)
    for ch in w + ".": 
        ix = stoi[ch]

        print(context, ix)
        X.append(context)
        Y.append(ix)

        print("".join(itos[i] for i in context), '---->', itos[ix])
        context = context[1:] + [ix]   # it's like rolling window 

    print("---"*5)

## This code tells us how the single word is converting to character level inputs 

X = torch.tensor(X)
Y = torch.tensor(Y)
print("XShape", X.shape, "YShape", Y.shape)

emma
[0, 0, 0] 5
... ----> e
[0, 0, 5] 13
..e ----> m
[0, 5, 13] 13
.em ----> m
[5, 13, 13] 1
emm ----> a
[13, 13, 1] 0
mma ----> .
---------------
olivia
[0, 0, 0] 15
... ----> o
[0, 0, 15] 12
..o ----> l
[0, 15, 12] 9
.ol ----> i
[15, 12, 9] 22
oli ----> v
[12, 9, 22] 9
liv ----> i
[9, 22, 9] 1
ivi ----> a
[22, 9, 1] 0
via ----> .
---------------
ava
[0, 0, 0] 1
... ----> a
[0, 0, 1] 22
..a ----> v
[0, 1, 22] 1
.av ----> a
[1, 22, 1] 0
ava ----> .
---------------
isabella
[0, 0, 0] 9
... ----> i
[0, 0, 9] 19
..i ----> s
[0, 9, 19] 1
.is ----> a
[9, 19, 1] 2
isa ----> b
[19, 1, 2] 5
sab ----> e
[1, 2, 5] 12
abe ----> l
[2, 5, 12] 12
bel ----> l
[5, 12, 12] 1
ell ----> a
[12, 12, 1] 0
lla ----> .
---------------
sophia
[0, 0, 0] 19
... ----> s
[0, 0, 19] 15
..s ----> o
[0, 19, 15] 16
.so ----> p
[19, 15, 16] 8
sop ----> h
[15, 16, 8] 9
oph ----> i
[16, 8, 9] 1
phi ----> a
[8, 9, 1] 0
hia ----> .
---------------
XShape torch.Size([32, 3]) YShape torch.Size([32])


In [113]:
# Now we have created a input data, let's create an embedding to store this inupts and outputs: 
## Here we have 27 unique characters and we are going to embed in low dimensional space. 

### Let's create a two dimensional embeddings (we are initializing randomly)
C = torch.randn( (27, 2) )  # 27 characters and 2 dimensional embeddings 

### Because it's a random number if you want to get the embedding for character "e", just get the index of "e" is 5 and do the indexing in C C[5] value is the
### -- embedding. Later this embeddings will learned by back-propagation. Initially this is just a random numbers. 

# Embedding of character "e"
print(C[stoi["m"]]) 

tensor([-0.2882,  0.2658])


In [114]:
## But the pytorch indexing is super powerfull, we can give two dimenional array to indexing, it can index. 
## It means we don't need a for loop to iterate over "X"(input), we can just give the "X" to the tensor and we get all the indexed elements.  

emb = C[X]  # X is the input of the model 
print(emb.shape)

## 32 is row of X, 3 is the dimension(column) of X, 2 is just we retrieved 2 len embeddings.  

torch.Size([32, 3, 2])


In [115]:
emb[0]

tensor([[0.1621, 0.0919],
        [0.1621, 0.0919],
        [0.1621, 0.0919]])

In [116]:
## Here we are having 2 dimensional space and 3 embeddings (block size) | Each character has 2 dimensional embeddings and we are taking 3 character to find the next character so input is 6 
W1 = torch.randn( (emb[0].numel(), 100))  # 6 is just a 3 character + 2 size embeddings, 100 is the next layer neuron 
b1 = torch.randn( 100 )


In [117]:
# our aim is multiply emb@W + b but here the shape of emb and W is different 

torch.cat(torch.unbind(emb, 1), dim = 1).shape

torch.Size([32, 6])